<a href="https://colab.research.google.com/github/Sandwhaletree/Mai_Finlab/blob/main/%E8%82%A1%E7%A5%A8%E8%B2%A1%E5%8B%99%E5%88%86%E6%9E%90/000_Crawing_publicinfo_0725.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🔥🔥🔥公開資訊-快速查詢公司財務報表**

In [1]:
import os
os.chdir('/content/drive/MyDrive/colab_cloud/finlab')

In [2]:
import requests
import pandas as pd
import datetime

## **👉A.快速查詢公司代碼&公司資訊**

In [3]:
def company_info(): #公司資訊

  #設計4個空變數，之後要呼叫4張公司資料表
  df1 = []  #company_info_sii
  df2 = []  #company_info_otc
  df2 = []  #company_info_rotc
  df4 = []  #company_info_pub

  #產生4種市場之公司基本資料
  markets = ["sii","otc","rotc","pub"]

  for market in markets:
    res =requests.get(f"https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK={market}&step=1&firstin=1") #透過網址把抓到的資料放到res
    df = pd.read_html(res.text)[0] #把需要的資料放在df
    filt = df[df['公司代號']!= '公司代號'] #利用欄位塞選，濾掉重複性的標題
    filt.insert(1,'市場別',market)
    # filt.set_index('公司代號', inplace=True) #將公司代號欄位，直接設計成index

    #把資料分別儲存至空變數中
    if market == "sii":
      df1 = filt
    elif market == "otc":
      df2 = filt
    elif market == "rotc":
      df3 = filt
    else:
      df4 = filt

  #建立空字典
  result={}

  #讀取df1~df4，並取出需要之資料
  for df in [df1,df2,df3,df4]:
    for index,col in df.iterrows():
      #key: value1、value2...、value2...
      result[col['公司代號']]=(col['營利事業統一編號'],col['公司簡稱'],col['市場別'],col['產業類別'],col['公司網址'])
  #print(result) #檢查1

  return result

In [4]:
def company_id (keys,result): #公司代碼
  # 建立空清單來存放結果
  values = []

  # 遍歷清單中的所有鍵值
  for key in keys:
      # 使用get方法查詢字典中的值
      value = result.get(key)
      # 將值加入結果清單中
      values.append(key)
      values.append(value)
  # 現在，您可以使用values列表查詢所有鍵值的值
  return values

In [5]:
def company_dataframe(values):
  # 建立空清單，將每個元組的數據存儲在清單中
  data = []

  # 遍歷清單中的每個元素
  for element in values:
      # 如果元素是元組，則加入清單中
      if isinstance(element, tuple):
          data.append(element)

  # 將清單轉換為DataFrame
  df = pd.DataFrame(data, columns=['id','name', 'exchange', 'industry', 'website'])
  return df

In [7]:
#欲查詢之股票
keys = input('輸入欲查詢之股票代號(用逗號分隔): ').split(',')

result = company_info() #return result
values = company_id(keys,result) # return values
df = company_dataframe(values) # return df
df

輸入欲查詢之股票代號(用逗號分隔): 2330


KeyError: ignored

##### **簡單舉例**

In [ ]:
df1=pd.DataFrame([ [1,'1_com','1_rev'],[2,'2_com','2_rev'] ], columns = ['id', 'name', 'rev'])
df2=pd.DataFrame([ [3,'3_com','3_rev'],[4,'4_com','4_rev'] ], columns = ['id', 'name', 'rev'])
df3=pd.DataFrame([ [5,'5_com','5_rev'],[6,'6_com','6_rev'] ], columns = ['id', 'name', 'rev'])
df4=pd.DataFrame([ [7,'7_com','7_rev'],[8,'8_com','8_rev'] ], columns = ['id', 'name', 'rev'])

In [ ]:
# 建立空字典
result = {}

# 遍歷每個dataframe
for df in [df1, df2, df3, df4]:
  print('df:',df)
  # 遍歷dataframe中的每個元素
  for index, row in df.iterrows():
    # 將index設為key，第一欄設為value，並加入字典中
    result[row['id']] = (row['name'],row['rev'])
    print('result:',result)

print(result)

In [ ]:
# 建立查詢鍵值的清單
keys = [2,7,6]

# 建立空清單來存放結果
values = []

# 遍歷清單中的所有鍵值
for key in keys:
    # 使用get方法查詢字典中的值
    value = result.get(key)
    # 將值加入結果清單中
    values.append(key)
    values.append(value)
# 現在，您可以使用values列表查詢所有鍵值的值
values

## **👉B.快速查詢上市、上櫃、興櫃與公發公司之財務報表**
  * 1)損益表
  * 2)資產負債表
  * 3)現金流量表
  * 4)營業分析表

In [ ]:
#輸入關鍵字

#市場 #market
market = int(input('請輸入欲查詢之市場數字-1)sii上市 2)otc上櫃 3)rotc興櫃 4)pub公開發行: '))
if market == 1:
    market = 'sii'
elif market == 2 :
    market = 'otc'
elif market == 3 :
    market = 'rotc'
elif market == 4 : #營業分析表
    market = 'pub'
else:
    print('type does not match')

#財報類型 #type
type = int(input('請輸入欲查詢之表格數字-1)損益表 2)資產負債表 3)現金流量表 4)營業分析表: '))
if type == 1: #損益表
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
elif type == 2 : #資產負債表
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
elif type == 3 : #現金流量表
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb20'
elif type == 4 : #營業分析表
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
else:
    print('type does not match')

#年份 #year
year = int(input('請輸入欲查詢年份: '))
if year >= 1000:
    year -= 1911

#季節 #season
season = input('請輸入欲查詢之季節:')

請輸入欲查詢之市場數字-1)sii上市 2)otc上櫃 3)rotc興櫃 4)pub公開發行: 2
請輸入欲查詢之表格數字-1)損益表 2)資產負債表 3)現金流量表 4)營業分析表: 2
請輸入欲查詢年份: 2020
請輸入欲查詢之季節:4


In [ ]:
#網站連結

res = requests.post(url, {
                        'encodeURIComponent':1,
                        'step':1,
                        'firstin':1,
                        'off':1,
                        'TYPEK': market,            #input market(str)
                        'year':str(year),           #input market(str)
                        'season':str(season),
                        })
#把資料分別儲存至空變數中
if market == "sii" :
  if type == 1 or type == 2 or type == 3:  #1)損益表 2)資產負債表 3)現金流量表 #[0]表頭
    df_bank = pd.read_html(res.text)[1]      #[1]=銀行
    df_sec = pd.read_html(res.text)[2]       #[2]=證券
    df_normal = pd.read_html(res.text)[3]    #[3]=一般
    df_fh = pd.read_html(res.text)[4]        #[4]=金控
    df_ins = pd.read_html(res.text)[5]       #[5]=保險
    df_rare = pd.read_html(res.text)[6]      #[6]=異業
  else: #4)營業分析表
    df_all = pd.read_html(res.text)[0]  #[0]=全部行業；[1]=附註
    df_all.columns = df_all.iloc[0]
    filt_all= df_all[df_all['公司代號'] != '公司代號'] #濾掉重複之表頭

elif market == "otc" :
  if type == 1 or type == 2 or type == 3:  #1)損益表 2)資產負債表 3)現金流量表 [0]表頭
    df_sec = pd.read_html(res.text)[1]  #[1]=證券
    df_normal = pd.read_html(res.text)[2]  #[2]=一般
  else: #4)營業分析表
    df_all = pd.read_html(res.text)[0]  #[0]=全部行業；[1]=附註
    df_all.columns = df_all.iloc[0]
    filt_all= df_all[df_all['公司代號'] != '公司代號'] #濾掉重複之表頭

elif market == "rotc" :
  if type == 1 or type == 2 or type == 3:  #1)損益表 2)資產負債表 3)現金流量表 #[0]=表頭
    df_bank = pd.read_html(res.text)[1]      #[1]=銀行
    df_sec = pd.read_html(res.text)[2]      #[2]=證券
    df_normal = pd.read_html(res.text)[3]      #[3]=一般
    df_ins = pd.read_html(res.text)[4]      #[4]=保險
  else: #4)營業分析表
    df_all = pd.read_html(res.text)[0]  #[0]=全部行業；[1]=附註
    df_all.columns = df_all.iloc[0]
    filt_all= df_all[df_all['公司代號'] != '公司代號'] #濾掉重複之表頭
else:
  if type == 1 or type == 2 or type == 3:  #1)損益表 2)資產負債表 3)現金流量表
    df_bank = pd.read_html(res.text)[1]  #[1]=銀行
    df_sec = pd.read_html(res.text)[2]  #[2]=證券
    df_normal = pd.read_html(res.text)[3]  #[3]=一般
    df_fh = pd.read_html(res.text)[4]  #[4]=金控
    df_ins = pd.read_html(res.text)[5]  #[5]=保險
  else: #4)營業分析表
    df_all = pd.read_html(res.text)[0]  #[0]=全部行業；[1]=附註
    df_all.columns = df_all.iloc[0]
    filt_all= df_all[df_all['公司代號'] != '公司代號'] #濾掉重複之表頭

In [ ]:
# df_2

In [ ]:
#設計函數financial_statement(year, season)
def financial_statement(market, year, season):
    res = requests.post(url, {
                            'encodeURIComponent':1,
                            'step':1,
                            'firstin':1,
                            'off':1,
                            'TYPEK': market,
                            'year':str(year),
                            'season':str(season),
                           })
    #把資料分別儲存至空變數中
    if market == "sii" or market == "rotc" or market == "pub" :
      if type == 1 or type == 2 or type == 3:  #1)損益表 2)資產負債表 3)現金流量表
        df = pd.read_html(res.text)[3] #df[0]=表頭；[1]=銀行；[2]=證券；[3]=一般；[4]=金控；[5]=保險；[6]=異業
      else:   #4)營業分析表
        df = pd.read_html(res.text)[2]
    else:
      if type == 1 or type == 2 or type == 3:  #1)損益表 2)資產負債表 3)現金流量表
        df = pd.read_html(res.text)[3] #df[0]=表頭；[1]=銀行；[2]=證券；[3]=一般；[4]=金控；[5]=保險；[6]=異業
    else:   #4)營業分析表
        df = pd.read_html(res.text)[2]

    df.insert(1,'年度', year)
    df.insert(2,'季別', season)

    #不同財報選取表格不同

    #1)損益表 2)資產負債表 3)現金流量表
    if type == 1 or type == 2 or type == 3:
      df = pd.read_html(res.text)[3] #df[0]=表頭；[1]=銀行；[2]=證券；[3]=一般；[4]=金控；[5]=保險；[6]=異業
    #4)營業分析表
    else:
      df = pd.read_html(res.text)[2]

    filt= df[df['公司代號'] != '公司代號'] #濾掉重複之表頭
    filt.set_index('公司名稱', inplace=True)
    return filt

In [ ]:
year = int(input('請輸入欲查詢年份: '))
if year >= 1000:
    year -= 1911

#2.財報類型 #type
type = int(input('請輸入欲查詢之表格數字-1)損益表 2)資產負債表 3)現金流量表 4)營業分析表: '))

if type == 1: #損益表
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
elif type == 2 : #資產負債表
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
elif type == 3 : #現金流量表
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb20'
elif type == 4 : #營業分析表
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
else:
    print('type does not match')

#3.季節 #season
season = input('請輸入欲查詢之季節:')

df = financial_statement(year, season)
df

請輸入欲查詢年份: 2020
請輸入欲查詢之表格數字-1)損益表 2)資產負債表 3)現金流量表 4)營業分析表: 1
請輸入欲查詢之季節:4


,公司代號,營業收入,營業成本,原始認列生物資產及農產品之利益（損失）,生物資產當期公允價值減出售成本之變動利益（損失）,營業毛利（毛損）,未實現銷貨（損）益,已實現銷貨（損）益,營業毛利（毛損）淨額,營業費用,...,其他綜合損益（淨額）,合併前非屬共同控制股權綜合損益淨額,本期綜合損益總額,淨利（淨損）歸屬於母公司業主,淨利（淨損）歸屬於共同控制下前手權益,淨利（淨損）歸屬於非控制權益,綜合損益總額歸屬於母公司業主,綜合損益總額歸屬於共同控制下前手權益,綜合損益總額歸屬於非控制權益,基本每股盈餘（元）
公司名稱,,,,,,,,,,,,,,,,,,,,,
台泥,1101,114367247,76779667,--,--,37587580,--,--,37587580,6245763,...,-2495498,--,25495895,25099309,--,2892084,22708608,--,2787287,4.32
亞泥,1102,78240880,54911404,--,--,23329476,0,--,23329476,3659338,...,-1189331,--,17584476,14710486,--,4063321,13255580,--,4328896,4.70
嘉泥,1103,2058417,2202158,--,--,-143741,--,--,-143741,603155,...,300229,--,2138223,1764366,--,73628,2051467,--,86756,2.74
環泥,1104,5426217,4519186,--,--,907031,--,--,907031,413889,...,79230,--,1339025,1247252,--,12543,1326470,--,12555,1.91
幸福,1108,4530172,3813497,--,--,716675,--,--,716675,260241,...,4426,--,396695,392269,--,0,396695,--,0,0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
新麗,9944,2347804,1716754,--,--,631050,--,--,631050,535495,...,-32391,--,57975,75447,--,14919,47570,--,10405,0.69
潤泰新,9945,19084589,15145020,--,--,3939569,--,--,3939569,1854562,...,25746073,--,35021465,8557651,--,717741,34343978,--,677487,5.88
三發地產,9946,1491772,1055716,--,--,436056,--,--,436056,226481,...,7842,--,190817,182975,--,--,190817,--,--,0.57


In [ ]:
df.query("公司代號 in [2365,2401,2596,2736,4109,5902,5905,6116,6291,8047] ")

,公司代號,營業收入,營業成本,原始認列生物資產及農產品之利益（損失）,生物資產當期公允價值減出售成本之變動利益（損失）,營業毛利（毛損）,未實現銷貨（損）益,已實現銷貨（損）益,營業毛利（毛損）淨額,營業費用,...,其他綜合損益（淨額）,合併前非屬共同控制股權綜合損益淨額,本期綜合損益總額,淨利（淨損）歸屬於母公司業主,淨利（淨損）歸屬於共同控制下前手權益,淨利（淨損）歸屬於非控制權益,綜合損益總額歸屬於母公司業主,綜合損益總額歸屬於共同控制下前手權益,綜合損益總額歸屬於非控制權益,基本每股盈餘（元）
公司名稱,,,,,,,,,,,,,,,,,,,,,
昆盈,2365,1653269,1137698,--,--,515571,--,--,515571,353776,...,-241637,--,-94732,146236,--,669,-95331,--,599,0.64
凌陽,2401,6414140,3489044,--,--,2925096,--,--,2925096,2408966,...,5718,--,624545,323403,--,295424,326913,--,297632,0.55
彩晶,6116,19553306,14713779,--,--,4839527,-27,--,4839554,2130575,...,516401,--,4351497,3847904,--,-12808,4364305,--,-12808,1.25


## **👉0.公司基本資料**

* 公司基本資料，MOPS主要查詢網站如下
 * [sii-上市公司基本資料](https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK=sii&step=1&firstin=1)
 * [otc-上櫃公司基本資料](https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK=otc&step=1&firstin=1)
 * [rotc-興櫃公司基本資料](https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK=otc&step=1&firstin=1)
 * [pub-公開發行公司基本資料](https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK=otc&step=1&firstin=1)

* 缺點:沒有董監事持股明細資料


### **💡0-1.4種公司基本資料類型EXCEL下載，並存至雲端路徑**

In [ ]:
#確保目前路徑為雲端finlab資料夾
import os
os.chdir('/content/drive/MyDrive/colab_cloud/finlab')

#建立新資料夾"公司基本資料查詢"，並將路徑移至資料夾
if not os.path.isdir('公司基本資料查詢'): # 確認是否有jobs_csv資料夾  沒有則返回Ture
  os.mkdir('公司基本資料查詢') # 建立jobs_csv資料夾
  print('建立公司基本資料查詢資料夾完成')
else:
  print('已有公司基本資料查詢資料夾')
os.chdir(f'./公司基本資料查詢')
print('--------------------------')

#一次產生4種類型公司基本資料
markets = ["sii","otc","rotc","pub"]

for market in markets:
  res =requests.get(f"https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK={market}&step=1&firstin=1") #透過網址把抓到的資料放到res
  df = pd.read_html(res.text)[0] #把需要的資料放在df
  filt = df[df['公司代號']!= '公司代號'] #利用欄位塞選，濾掉重複性的標題
  filt.set_index('公司代號', inplace=True) #將公司代號欄位，直接設計成index

  #客製化檔名
  date = str(datetime.date.today()) #今日日期戳印
  filt.to_excel(f'./{date}_公司基本資料_{market}.xlsx')
  print('完成下載',f'{date}_公司基本資料_{market}.xlsx')

建立公司基本資料查詢資料夾完成
--------------------------
完成下載 2022-12-14_公司基本資料_sii.xlsx
完成下載 2022-12-14_公司基本資料_otc.xlsx
完成下載 2022-12-14_公司基本資料_rotc.xlsx
完成下載 2022-12-14_公司基本資料_pub.xlsx


### **💡0-2.4種公司基本資料類型儲存至DataFrame，並呼叫使用**

In [ ]:
def Company_info():

  #設計4個空變數，之後要呼叫4張公司資料表
  company_info_sii = []
  company_info_otc = []
  company_info_rotc = []
  company_info_pub = []

  #一次產生4張公司基本資料
  markets = ["sii","otc","rotc","pub"]

  for market in markets:
    res =requests.get(f"https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK={market}&step=1&firstin=1") #透過網址把抓到的資料放到res
    df = pd.read_html(res.text)[0] #把需要的資料放在df
    filt = df[df['公司代號']!= '公司代號'] #利用欄位塞選，濾掉重複性的標題
    filt.set_index('公司代號', inplace=True) #將公司代號欄位，直接設計成index

    #把資料分別儲存至空變數中
    if market == "sii":
      company_info_sii = filt
    elif market == "otc":
      company_info_otc = filt
    elif market == "rotc":
      company_info_rotc = filt
    else:
      company_info_pub = filt

  return company_info_sii,company_info_otc,company_info_rotc,company_info_pub

In [ ]:
#呼叫4種類型公司資料表
company_info_sii,company_info_otc,company_info_rotc,company_info_pub = Company_info()

In [ ]:
#上市公司981家
company_info_sii.head()

,公司名稱,公司簡稱,產業類別,外國企業註冊地國,住址,營利事業統一編號,董事長,總經理,發言人,發言人職稱,...,英文簡稱,英文通訊地址,傳真機號碼,電子郵件信箱,公司網址,投資人關係聯絡人,投資人關係聯絡人職稱,投資人關係聯絡電話,投資人關係聯絡電子郵件,公司網站內利害關係人專區網址
公司代號,,,,,,,,,,,,,,,,,,,,,
1101,臺灣水泥股份有限公司,台泥,水泥工業,－,台北市中山北路2段113號,11913502,張安平,程耀輝,黃健強,資深副總經理,...,TCC,"No.113, Sec.2, Zhongshan N. Rd.,Taipei City 10...",(02)2531-6529,finance@taiwancement.com,http://www.taiwancement.com,張佳琪,主任,02-25317099分機20358,ir@taiwancement.com,http://www.taiwancement.com/tw/csr/csr5-1.html
1102,亞洲水泥股份有限公司,亞泥,水泥工業,－,台北市大安區敦化南路2段207號30、31樓,03244509,徐旭東,李坤炎,周維崑,副總經理,...,ACC,"30-31F., No.207, Sec. 2, Dunhua S. Rd., Da' an...",02-2378-5191,service@acc.com.tw,www.acc.com.tw,陳韋仲,投資人關係副經理,02 27338000 ext.8336,ir@acc.com.tw,http://www.acc.com.tw/tw/stakeholder/stakehold...
1103,嘉新水泥股份有限公司,嘉泥,水泥工業,－,台北市中山北路2段96號,11892801,張剛綸,王立心,王立心,總經理,...,CHC,"No.96, Sec. 2, Zhongshan N. Rd., Zhongshan Dis...",(02)2511-4002,ir@chcgroup.com.tw,www.chcgroup.com.tw,王立心,總經理,(02)2551-5211#243,ir@chcgroup.com.tw,www.chcgroup.com.tw/index.php?route=system/sys...
1104,環球水泥股份有限公司,環泥,水泥工業,－,台北市南京東路二段125號10樓,07568009,侯博義,侯智升,楊宗仁,副總經理,...,UCC,"10th F1., 125 Nanking E.Rd Sec. 2,Taipei Taiwan",02-25075870,ucc@ucctw.com,www.ucctw.com,楊宗仁,副總經理,02-2507-7801,p1590@ucctw.com,http://www.ucctw.com/ucctw/index.php?option=co...
1108,幸福水泥股份有限公司,幸福,水泥工業,－,台北市松江路237號15樓,40601248,陳兩傳,陳兩傳,溫秀菊,財務部經理,...,LUCKY CEMENT,"15TH FL.,NO.237,SUNG KIANG RD.,TAIPEI,TAIWAN,R...",25048672,"wendy@luckygrp,com.tw",www.luckygrp.com.tw,溫秀菊,財務部經理,25092188,wendy@luckygrp.com.tw,http://www.luckygrp.com.tw/tw/index.asp?au_id=...


In [ ]:
#上櫃公司 805家
company_info_otc.head(10)

,公司名稱,公司簡稱,產業類別,外國企業註冊地國,住址,營利事業統一編號,董事長,總經理,發言人,發言人職稱,...,英文簡稱,英文通訊地址,傳真機號碼,電子郵件信箱,公司網址,投資人關係聯絡人,投資人關係聯絡人職稱,投資人關係聯絡電話,投資人關係聯絡電子郵件,公司網站內利害關係人專區網址
公司代號,,,,,,,,,,,,,,,,,,,,,
1240,茂生農經股份有限公司,茂生農經,農業科技,－,台北市和平西路一段三十號二樓,18795706,吳清德,吳清德,林信鴻,管理部經理,...,MORNSUN,"2F.,No.30,Sec. 1,Heping W.Rd.,Zhongzheng Dist....",02-23640694,bedford@morn-sun.com.tw,https://www.morn-sun.com.tw/,林信鴻,管理部經理,02-23671162#106,bedford@morn-sun.com.tw,https://www.morn-sun.com.tw/
1258,其祥生物科技控股有限公司,其祥-KY,食品工業,KY 開曼群島,台北市中山區松江路51號8樓,53028230,王建山,陳偉業,王建山,董事長,...,Kee Song,"8F., No.51, Songjiang Rd., Zhongshan Dist.Taip...",(02)2503-4933,investor@keesong.com,http://keesongbiotech.com/,陳英傑,財務經理,(02)2518-4933,investor@keesong.com,http://keesongbiotech.com/intstakeholders/
1259,安心食品服務股份有限公司,安心,觀光事業,－,台北市大安區仁愛路3段136號2樓201室,23928945,林建元,高順興,施啟寅,協理,...,AN-SHIN,"Room201, 2F., No. 136,Sec. 3, Ren'ai Rd.Da’an ...",(02)2567-1692,ir@mos.com.tw,http://www.mos.com.tw,張貴梅,副理,(02)2567-5001,ir@mos.com.tw,http://www.mos.com.tw/investWindow.php
1264,德麥食品股份有限公司,德麥,食品工業,－,新北市五股區五權五路31號,23420497,林雪嬌,吳文欽,邱俊榮,財務經理,...,TEHMAG,"No.31,Wuquan 5th Rd.,Wugu Dist.,New Taipei Cit...",(02)2298-2263,ir@tehmag.com.tw,http://www.tehmag.com.tw,邱俊榮,財務經理,(02)2298-1347,willychiou@tehmag.com.tw,http://www.tehmag.com.tw/webnews/view.php?sn=3548
1268,漢來美食股份有限公司,漢來美食,觀光事業,－,高雄市前金區成功一路266號21樓之1,80352343,賴宗成,林淑婷,林淑婷,總經理,...,Hi-Lai Foods,"21F-1, No.266, Cheng-Kung 1st Road,80146 Kaohs...",(07)215-5956,stock@hilai-foods.com,www.hilai-foods.com,柯柏良,財務長,(07)213-5788,stock@hilai-foods.com,www.hilai-foods.com
1336,台翰精密科技股份有限公司,台翰,電子零組件業,－,桃園市龜山區民生北路一段568號1樓,22589593,蔡鎮隆,蔡鎮隆,張誌仁,執行副總經理,...,TAIHAN,"1F., No. 568, Sec. 1, Minsheng N. Rd., Guishan...",03-3112035,tony@thpt.com.tw,www.thpt.com.tw,張誌仁,執行副總經理,03-3112025,tony@thpt.com.tw,http://www.thpt.com.tw/parnerstocknews1.html
1565,精華光學股份有限公司,精華,生技醫療業,－,新北市汐止區大同路一段276之2號4、5樓,22195654,陳明賢,周育然,周華玲,稽核室副理,...,St.Shine,"4, 5F, No. 276-2, Sec. 1, Ta Tung Rd,,Hsi Chih...",(02)2647-9369,s0505@stshine.com.tw,http://www.stshine.com.tw,周華玲,稽核室副理,(02)2647-1578,s0505@stshine.com.tw,http://www.stshine.com.tw
1569,濱川企業股份有限公司,濱川,電腦及週邊設備業,－,高雄市大寮區大發工業區興業路二號,76404919,蕭欽銘,蕭欽群,李璠,副總經理,...,Bin Chuan,2 Shing Yeh Road Ta-fa Industrial ZoneKoahsiun...,07-7871515,fan_lee@hamagawa.com,www.hamagawa.com,李璠,副總經理,07-7871521分機16,investors@hamagawa.com,http://www.hamagawa.com/
1570,力肯實業股份有限公司,力肯,電機機械,－,新北市八里區博物館路81號,36253534,溫銘漢,溫銘漢,溫銘漢,董事長兼總經理,...,De Poan,"NO.81, MUSEUM ROADBa-Li District, New Taipei C...",(02)2619-5618,wilson-ko@depoan.com,www.depoan.com,吳曼群,總管理處協理,(02)2619-5619,grace.wu@depoan.com,http://www.depoan.com/about_03.php?ef=1


In [ ]:
#興櫃公司 346家
company_info_rotc.head()

,公司名稱,公司簡稱,產業類別,外國企業註冊地國,住址,營利事業統一編號,董事長,總經理,發言人,發言人職稱,...,英文簡稱,英文通訊地址,傳真機號碼,電子郵件信箱,公司網址,投資人關係聯絡人,投資人關係聯絡人職稱,投資人關係聯絡電話,投資人關係聯絡電子郵件,公司網站內利害關係人專區網址
公司代號,,,,,,,,,,,,,,,,,,,,,
1260,富味鄉食品股份有限公司,富味鄉,食品工業,－,台北市大安區市民大道四段102號11樓,12467902,陳昶宏,劉兆華,陳淑紋,副總經理,...,FLAVOR,"11F., No. 102, Sec. 4, Civic Blvd., Da'an Dist...",(02)2750-6777,flavor@flavor.com.tw,www.flavor.com.tw,－,－,－,－,www.flavor.com.tw
1269,乾杯股份有限公司,乾杯,觀光事業,－,台北市中正區忠孝東路二段88號11樓,70443909,平出莊司,平出莊司,廖佳怡,董事長室經理,...,KANPAI,"11F., No.88, Sec. 2, Zhongxiao E. Rd., Zhongzh...",(02)3322-2696,kanpai_otc@kanpai.com.tw,www.kanpai.com.tw,－,－,－,－,www.kanpai.com.tw
1271,晨暉生物科技股份有限公司,晨暉生技,生技醫療業,－,台北市內湖區行愛路139號,28708250,李家榮,潘佳岳,曾騰賢,副總經理,...,SunWay,"No.139, Xing'ai Rd., Neihu DistrictTaipei City...",(02)2792-9566,ir@sunway.cc,www.sunway.cc,－,－,－,－,www.sunway.cc
1293,利統股份有限公司,利統,食品工業,－,新北市汐止區康寧街169巷31-1號13樓之1,89955491,張天鴻,張天鴻,蔡沛協,財務長,...,Lytone,"13F-1, No. 31-1, Ln. 169, Kangning St., Xizhi ...",(02)2694-0121,terry.tsai@lytone.com,http://www.lytone.com,－,－,－,－,http://www.lytone.com
1294,漢田生技股份有限公司,漢田生技,食品工業,－,彰化縣員林市中央里中山路一段60號,28814153,李雨霖,陳岳甫,劉騏豪,財務長,...,HTB,"No.60, Sec. 1, Zhongshan Rd., Yuanlin City,Cha...",04-8366214,IR@htbios.com,https://www.htbios.com/,－,－,－,－,無


In [ ]:
#興櫃公司 300家
company_info_pub.head()

,公司名稱,公司簡稱,產業類別,外國企業註冊地國,住址,營利事業統一編號,董事長,總經理,發言人,發言人職稱,...,英文簡稱,英文通訊地址,傳真機號碼,電子郵件信箱,公司網址,投資人關係聯絡人,投資人關係聯絡人職稱,投資人關係聯絡電話,投資人關係聯絡電子郵件,公司網站內利害關係人專區網址
公司代號,,,,,,,,,,,,,,,,,,,,,
000104,臺銀綜合證券股份有限公司,臺銀證券,證券,－,臺北市重慶南路1段58號4~9樓,28428390,魏江霖,謝秀賢,林溫琴,副總經理,...,BankTaiwan Sec.,"4~9F., No.58, Sec. 1, Chongching S. Rd., Jhong...",(02)2371-7121,sec00301@twfhcsec.com.tw,http://www.twfhcsec.com.tw,－,－,－,－,不適用
000111,臺灣中小企銀,臺灣企銀,證券,－,台北市塔城街30號,14329223,林謙浩,張志堅,林昌育,副總經理,...,TBB,"NO.30 Ta Cheng ST. Taipei,Taiwan,R.O.CNO.30 Ta...",02-25509987,b035@mail.tbb.com.tw,www.tbb.com.tw,－,－,－,－,配合本行網頁更新，預計104年10月底前設立
000116,日盛證券股份有限公司,日盛證券,證券,－,台北市南京東路2段111號3、4樓,03098908,程明乾,顏謝雨潤(代理總經理),程明乾,董事長,...,JIHSUN SCL,"3、4F, No.111 Sec.2 Nan King E. Rd., TaipeiTaiw...",(02)2517-2442,net-service@jihsun.com.tw,www.jihsun.com.tw,－,－,－,－,NaN
000218,亞東證券股份有限公司,亞東證券,證券,－,新北市板橋區新站路16號13、14樓,36552651,杜金森,王珮恩,王珮恩,總經理,...,OSC,"13、14F,NO 16, Xinzhan Rd.,Banqiao Dist.New Tai...",(02)77531845,admin@osc.com.tw,http://www.osc.com.tw,－,－,－,－,http://www.osc.com.tw/index_body.asp
000511,富隆證券股份有限公司,富隆,證券,－,北市南京東路三段287號3樓,22956788,鄭淑華,宋世敏,宋世敏,總經理,...,FULLONG,"3F.,No.287,Sec.3,Nanjing E.Rd.,Sungshan Chiu,T...",27139123,webadmin@fullong.com.tw,http://www.fullong.com.tw,－,－,－,－,http://www.fullong.com.tw


## **👉1.董監事持股餘額明細**
* 董監事持股餘額明細資料，MOPS主要查詢網站如下
 * [董監事持股餘額明細資料-以2330台積電為例](https://mops.twse.com.tw/mops/web/stapap1)
 * [參考影片-Python -MOPS董監事持股餘額明細資料之爬蟲教學](https://www.youtube.com/watch?v=wFqeQ4Ji9GQ)
  * [參考影片-Python -MOPS董監事持股 批次下載與Excel轉檔](https://www.youtube.com/watch?v=DzMwQ5rN9DA)
  * [參考影片-Python -以pandas之merge方法來合併MOPS資料](https://www.youtube.com/watch?v=gem-uSFqUFM)

* 缺點:沒有公司名稱&董監事持股比例搭配

### **💡1-1.董監事持股餘額明細資料EXCEL下載，並存至雲端路徑**

In [ ]:
#確保目前路徑為雲端finlab資料夾
import os
os.chdir('/content/drive/MyDrive/colab_cloud/finlab')

#建立新資料夾"公司基本資料查詢"，並將路徑移至資料夾
if not os.path.isdir('董監事持股餘額明細'): # 確認是否有jobs_csv資料夾  沒有則返回Ture
  os.mkdir('董監事持股餘額明細') # 建立jobs_csv資料夾
  print('建立"董監事持股餘額明細"夾完成')
else:
  print('已有董監事持股餘額明細資料夾')
os.chdir(f'./董監事持股餘額明細')
print('--------------------------')

#函數get_董監事持股餘額明細
def get_董監事持股餘額明細(co_id,year,month):

    url = 'https://mops.twse.com.tw/mops/web/ajax_stapap1'
    parameter = {'firstin': '1', 'co_id': str(co_id),
                 'year': str(year), 'month': str(month), }
    res = requests.post(url, data = parameter)
    df = pd.read_html(res.text)[3]

    排版 = {'公司代號':co_id,'年度': year,'月份': month, #抓出參數
            '職稱' :df[0],'姓名' :df[1],'選任時持股':df[2],'目前持股':df[3],
            '設質股數':df[4], '設質股數佔持股比例' :df[5],
            '內部關係人持股數':df[6],'內部關係人設質股數': df[7],'內部關係人設質比例':df[8]}

    table = pd.DataFrame(排版)
    filt = table[table['職稱'] != '職稱'] #濾掉每隔10列會重複的表頭
    filt.set_index('公司代號', inplace=True) #用公司代號當作index
    return filt

#取得股票代號清單、民國年度、月份
#股票清單:1, 1103, 1104, 1108, 1109, 1110, 1201, 1203, 1210

stock_list =list(map(int,input("請輸入股票代號： ").split(',')))  #抓取公司基本資料取得股票代號
year = int(input("請輸入抓取年份："))
month = int(input("請輸入抓取月份："))

print('抓取清單: ',stock_list,'\n','抓取年份：',year,'\n','抓取月份：',month,'\n')

data = pd.DataFrame() #建立pd.DataFrame格式之空集合
for co_id in stock_list:
    stock_holding = get_董監事持股餘額明細(co_id,year,month) #抓取每一間公司的董監明細
    data = pd.concat([data,stock_holding]) #把所有資料合併
    print(f'完成[{co_id}公司{year}年{month}月]之董監事持股餘額明細查詢')

#客製化檔名
date = str(datetime.date.today()) #今日日期戳印
data.to_excel(f'./{date}_董監事持股額明細_{year}年{month}月.xlsx')
print(f'{date}_董監事持股額明細_{year}年{month}月.xlsx，完成下載!!!')

已有董監事持股餘額明細資料夾
--------------------------
請輸入股票代號： 1101, 1102
請輸入抓取年份：111
請輸入抓取月份：10
抓取清單:  [1101, 1102] /n 抓取年份： 111 /n 抓取月份： 10 /n
完成[1101公司111年10月]之董監事持股餘額明細查詢
完成[1102公司111年10月]之董監事持股餘額明細查詢
2022-12-14_董監事持股額明細_111年10月.xlsx，完成下載!!!


### **💡1-2.董監事持股餘額明細儲存至DataFrame，並呼叫使用**

In [ ]:
sii = company_info_sii.index.tolist()
len(sii)

<class 'list'>


In [ ]:
otc = company_info_otc.index.tolist()
len(otc)

805

In [ ]:
pub = company_info_pub.index.tolist()
len(pub)

346

In [ ]:
rotc = company_info_rotc.index.tolist()
len(rotc)

300

In [ ]:
# 透過input公司代號去比對是否有在sii
co_id = input("請輸入公司代號：")

if co_id in sii:
  print("該公司存在於sii列表中")
elif co_id in otc:
  print("該公司存在於otc列表中")
elif co_id in rotc:
  print("該公司存在於rotc列表中")
else:
    print("該公司存在於pub列表中")

請輸入公司代號：1508
該公司存在於pub列表中


### **💡1-3.合併MOPS公司基本資料&董監事持股明細**

In [ ]:
#MOPS合併公司基本資料&董監事持股

year = int(input("請輸入抓取年份："))
month = int(input("請輸入抓取月份："))

# print('抓取年份：',year,'\n','抓取月份：',month,'\n')

#1.公司基本資料取得 #存至filt
market = "sii" #上市:sii、上櫃:otc、興櫃:rotc、公開發行:pub
res =requests.get(f"https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK={market}&step=1&firstin=1") #透過網址把抓到的資料放到res
df = pd.read_html(res.text)[0] #把需要的資料放在df
filt1 = df[df['公司代號']!= '公司代號'] #利用欄位塞選，濾掉重複性的標題
filt1.set_index('公司代號', inplace=True) #用公司代號當作index

#2.取得股票代號清單
抓取清單 = filt1.index.tolist()[1:3]#抓取公司基本資料取得股票代號

#3.設計函數get_董監事持股餘額明細 #存至filt
def get_董監事持股餘額明細(co_id,year,month):

    url = 'https://mops.twse.com.tw/mops/web/ajax_stapap1'
    parameter = {'firstin': '1', 'co_id': str(co_id),
                 'year': str(year), 'month': str(month), }
    res = requests.post(url, data = parameter)
    df = pd.read_html(res.text)[3]

    排版 = {'公司代號':co_id,'年度': year,'月份': month, #抓出參數
            '職稱' :df[0],'姓名' :df[1],'選任時持股':df[2],'目前持股':df[3],
            '設質股數':df[4], '設質股數佔持股比例' :df[5],
            '內部關係人持股數':df[6],'內部關係人設質股數': df[7],'內部關係人設質比例':df[8]}

    table = pd.DataFrame(排版)
    filt2 = table[table['職稱'] != '職稱'] #濾掉每隔10列會重複的表頭
    filt2.set_index('公司代號', inplace=True) #用公司代號當作index
    return filt2

#4.董監事持股餘額明細取得  #存至data
data = get_董監事持股餘額明細('1101',year, month) #【改年、月】
for i in 抓取清單:
    抓取=get_董監事持股餘額明細(i,year, month) #抓取每一間公司的董監明細 #【改年、月】
    data= pd.concat([data,抓取]) #把所有資料合併

#5.合併公司基本資料
mg=pd.merge(data, filt1, left_index = True, right_index = True)

#6.計算持股比率
mg.insert(7,'持股比率', pd.to_numeric(mg['目前持股'])/
                      (pd.to_numeric(mg['已發行普通股數或TDR原發行股數'])+
                        pd.to_numeric(mg['特別股(股)'])))

#6.儲存
import os
os.chdir('/content/drive/MyDrive/colab_cloud/finlab')
date = str(datetime.date.today())
mg.to_excel(f'./{date}_公司基本資料與董監事合併表_{year}年{month}月.xlsx')
print('完成下載',f'{date}_公司基本資料與董監事合併表_{year}年{month}月.xlsx')

請輸入抓取年份：111
請輸入抓取月份：5
抓取年份： 111 
 抓取月份： 5 

完成下載 ./2022-12-14_公司基本資料與董監事合併表_111年5月.xlsx


## **👉2.快速取得「全部」上市櫃公司之資產負債表、損益表、現金流量表**
 * 下方為資產負債表、損益表、現金流量表主要查詢網站
  * [公開資訊觀測站-綜合損益表](https://mops.twse.com.tw/mops/web/t163sb04)

 * 主要參考影片如下:
  * [Python爬蟲｜快速取得「全部」上市櫃公司之資產負債表、損益表、現金流量表](https://www.youtube.com/watch?v=FWF3KMj_AA8)

### **💡2-1.IFRS後-綜合損益表**

In [ ]:
#IFRS後-綜合損益表 #3/31為第四季財報終結日，最好4/1再爬資料
import requests
import pandas as pd


year_end = int(input("請輸入抓取結束年份："))
if year_end >= 1000:
    year_end -= 1911
year_gap = int(input("請輸入抓幾年資料："))

year_start = (year_end-year_gap)


#函數get_綜合損益表 #存至df
def get_綜合損益表(market,year,season): #定義函數
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    parameter = {'firstin': '1', 'TYPEK': market,
                 'year': str(year), 'season': str(season)}
    res = requests.post(url, data = parameter)

    #把資料分別儲存至空變數中
    if market == "sii" or market == "rotc" or market == "pub" :
      df = pd.read_html(res.text)[3]
    else:
      df = pd.read_html(res.text)[2]

    df.insert(1,'年度', year)
    df.insert(2,'季別', season)
    return df

#確保目前路徑
import os
os.chdir('/content/drive/MyDrive/colab_cloud/finlab')

#建立新資料夾"公司基本資料查詢"，並將路徑移至資料夾
if not os.path.isdir('綜合損益表'): # 確認是否有jobs_csv資料夾  沒有則返回Ture
  os.mkdir('綜合損益表') # 建立jobs_csv資料夾
  print('建立"綜合損益表"夾完成')
else:
  print('已有綜合損益表資料夾')
os.chdir(f'./綜合損益表')
print('--------------------------')

markets = ["sii","otc","rotc","pub"]
 #建立pd.DataFrame格式之空集合
data = pd.DataFrame()

for market in markets:
  print('目前market為:',market)
  for i in range(year_end, year_start, -1): #抓取110-101年
    IS_sheet=get_綜合損益表(market, i, 4) #將上市公司[110-101年]之第四季資料印出
    print('目前年度',i)
    data= pd.concat([data,IS_sheet]) #合併


  #客製化檔名
  date = str(datetime.date.today()) #今日日期戳印
  data.to_excel(f'./{date}_{market}綜合損益表_{year_start}-{year_end}.xlsx')
  print(f'{date}_{market}綜合損益表_{year_start}-{year_end}.xlsx，完成下載!!!')
  data = pd.DataFrame()

請輸入抓取結束年份：2020
請輸入抓幾年資料：3
已有綜合損益表資料夾
--------------------------
market: sii
i 109
第一次資料 4
i 108
第一次資料 4
i 107
第一次資料 4
2022-12-15_sii綜合損益表_106-109.xlsx，完成下載!!!
文檔完資料 4
market: otc
i 109
第一次資料 4
i 108
第一次資料 4
i 107
第一次資料 4
2022-12-15_otc綜合損益表_106-109.xlsx，完成下載!!!
文檔完資料 4
market: rotc
i 109
第一次資料 4
i 108
第一次資料 4
i 107
第一次資料 4
2022-12-15_rotc綜合損益表_106-109.xlsx，完成下載!!!
文檔完資料 4
market: pub
i 109
第一次資料 4
i 108
第一次資料 4
i 107
第一次資料 4
2022-12-15_pub綜合損益表_106-109.xlsx，完成下載!!!
文檔完資料 4


In [ ]:
url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
parameter = {'firstin': '1', 'TYPEK': 'rotc',
              'year': 110, 'season': 4}
res = requests.post(url, data = parameter)

In [ ]:
df = pd.read_html(res.text)[3]
df

,公司代號,公司名稱,營業收入,營業成本,原始認列生物資產及農產品之利益（損失）,生物資產當期公允價值減出售成本之變動利益（損失）,營業毛利（毛損）,未實現銷貨（損）益,已實現銷貨（損）益,營業毛利（毛損）淨額,...,其他綜合損益（淨額）,合併前非屬共同控制股權綜合損益淨額,本期綜合損益總額,淨利（淨損）歸屬於母公司業主,淨利（淨損）歸屬於共同控制下前手權益,淨利（淨損）歸屬於非控制權益,綜合損益總額歸屬於母公司業主,綜合損益總額歸屬於共同控制下前手權益,綜合損益總額歸屬於非控制權益,基本每股盈餘（元）
0,1260,富味鄉,4651294,3997799,--,--,653495,--,--,653495,...,-2909,--,208115,211024,--,--,208115,--,--,2.07
1,1269,乾杯,3079972,1510670,--,--,1569302,--,--,1569302,...,-5997,--,65253,71250,--,--,65253,--,--,3.83
2,1271,晨暉生技,380433,109323,--,--,271110,--,--,271110,...,946,--,170663,168760,--,957,169706,--,957,5.10
3,1293,利統,466942,272514,0,0,194428,--,--,194428,...,-382,--,36223,36605,--,--,36223,0,0,3.37
4,1294,漢田生技,240294,139266,--,--,101028,--,--,101028,...,--,--,31085,32558,--,-1473,32558,--,-1473,2.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,8465,德河,67949,57949,--,--,10000,--,--,10000,...,--,--,-4253,--,--,--,--,--,--,-0.21
294,8487,愛爾達,932376,788111,--,--,144265,--,--,144265,...,22,--,42113,--,--,--,--,--,--,2.08
295,8490,利得,41627,25555,--,--,16072,--,--,16072,...,0,--,-28606,--,--,--,--,--,--,-0.60
296,8492,兔將,196024,194063,--,--,1961,--,--,1961,...,70,--,-51460,-51530,--,--,-51460,--,--,-2.40


### **💡2-2.IFRS後-資產負債表**

In [ ]:
#IFRS後-資產負債表 #3/31為第四季財報終結日，最好4/1再爬資料
import requests
import pandas as pd


year_end = int(input("請輸入抓取結束年份："))
if year_end >= 1000:
    year_end -= 1911
year_gap = int(input("請輸入抓幾年資料："))

year_start = (year_end-year_gap)


#函數get_資產負債表 #存至df
def get_資產負債表(market,year,season): #定義函數
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05' #t163sb{04}，其中05:資產負債表；20:現金流量表 #【數字替換】
    parameter = {'firstin': '1', 'TYPEK': market,
                 'year': str(year), 'season': str(season)}
    res = requests.post(url, data = parameter)

    #把資料分別儲存至空變數中
    if market == "sii" or market == "rotc" or market == "pub" :
      df = pd.read_html(res.text)[3]
    else:
      df = pd.read_html(res.text)[2]

    df.insert(1,'年度', year)
    df.insert(2,'季別', season)
    return df

#確保目前路徑
import os
os.chdir('/content/drive/MyDrive/colab_cloud/finlab')

#建立新資料夾"資產負債表"，並將路徑移至資料夾
if not os.path.isdir('資產負債表'): # 確認是否有jobs_csv資料夾  沒有則返回Ture
  os.mkdir('資產負債表') # 建立jobs_csv資料夾
  print('建立"資產負債表"夾完成')
else:
  print('已有"資產負債表"資料夾')
os.chdir(f'./資產負債表')
print('--------------------------')

markets = ["sii","otc","rotc","pub"]
 #建立pd.DataFrame格式之空集合
data = pd.DataFrame()

for market in markets:
  print('目前market為:',market)
  for i in range(year_end, year_start, -1): #抓取110-101年
    IS_sheet=get_資產負債表(market, i, 4) #將上市公司[110-101年]之第四季資料印出
    print('目前年度:',i)
    data= pd.concat([data,IS_sheet]) #合併


  #客製化檔名
  date = str(datetime.date.today()) #今日日期戳印
  data.to_excel(f'./{date}_{market}資產負債表_{year_start}-{year_end}.xlsx')
  print(f'{date}_{market}資產負債表_{year_start}-{year_end}.xlsx，完成下載!!!')
  data = pd.DataFrame()
  print('---------------')

請輸入抓取結束年份：2020
請輸入抓幾年資料：3
建立"資產負債表"夾完成
--------------------------
目前market為: sii
目前年度 109
目前年度 108
目前年度 107
2022-12-15_sii資產負債表_106-109.xlsx，完成下載!!!
目前market為: otc
目前年度 109
目前年度 108
目前年度 107
2022-12-15_otc資產負債表_106-109.xlsx，完成下載!!!
目前market為: rotc
目前年度 109
目前年度 108
目前年度 107
2022-12-15_rotc資產負債表_106-109.xlsx，完成下載!!!
目前market為: pub
目前年度 109
目前年度 108
目前年度 107
2022-12-15_pub資產負債表_106-109.xlsx，完成下載!!!


### **💡2-3.IFRS後-現金流量**

In [ ]:
#IFRS後-現金流量表 #3/31為第四季財報終結日，最好4/1再爬資料
import requests
import pandas as pd


year_end = int(input("請輸入抓取結束年份："))
if year_end >= 1000:
    year_end -= 1911
year_gap = int(input("請輸入抓幾年資料："))

year_start = (year_end-year_gap)


#函數get_現金流量表 #存至df
def get_現金流量表(market,year,season): #定義函數
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb20' #t163sb{04}，其中05:資產負債表；20:現金流量表 #【數字替換】
    parameter = {'firstin': '1', 'TYPEK': market,
                 'year': str(year), 'season': str(season)}
    res = requests.post(url, data = parameter) #post()指定網址url，並發送資料data

    #把資料分別儲存至空變數中
    if market == "sii" or market == "rotc" or market == "pub" :
      df = pd.read_html(res.text)[3] #用read_html() 方法讀取指定的資料res.text
    else:
      df = pd.read_html(res.text)[2]

    df.insert(1,'年度', year)
    df.insert(2,'季別', season)
    return df

#確保目前路徑
import os
os.chdir('/content/drive/MyDrive/colab_cloud/finlab')

#建立新資料夾"現金流量表"，並將路徑移至資料夾
if not os.path.isdir('現金流量表'): # 確認是否有jobs_csv資料夾  沒有則返回Ture
  os.mkdir('現金流量表') # 建立jobs_csv資料夾
  print('建立"現金流量表"夾完成')
else:
  print('已有"現金流量表"資料夾')
os.chdir(f'./現金流量表')
print('--------------------------')

markets = ["sii","otc","rotc","pub"]
 #建立pd.DataFrame格式之空集合
data = pd.DataFrame()

for market in markets:
  print('目前market為:',market)
  for i in range(year_end, year_start, -1): #抓取110-101年
    IS_sheet=get_現金流量表(market, i, 4) #將上市公司[110-101年]之第四季資料印出
    print('目前年度:',i)
    data= pd.concat([data,IS_sheet]) #合併


  #客製化檔名
  date = str(datetime.date.today()) #今日日期戳印
  data.to_excel(f'./{date}_{market}現金流量表_{year_start}-{year_end}.xlsx')
  print(f'{date}_{market}現金流量表_{year_start}-{year_end}.xlsx，完成下載!!!')
  data = pd.DataFrame()
  print('---------------')

請輸入抓取結束年份：2020
請輸入抓幾年資料：3
已有"現金流量表"資料夾
--------------------------
目前market為: sii
目前年度: 109
目前年度: 108
目前年度: 107
2022-12-15_sii現金流量表_106-109.xlsx，完成下載!!!
---------------
目前market為: otc
目前年度: 109
目前年度: 108
目前年度: 107
2022-12-15_otc現金流量表_106-109.xlsx，完成下載!!!
---------------
目前market為: rotc
目前年度: 109
目前年度: 108
目前年度: 107
2022-12-15_rotc現金流量表_106-109.xlsx，完成下載!!!
---------------
目前market為: pub
目前年度: 109
目前年度: 108
目前年度: 107
2022-12-15_pub現金流量表_106-109.xlsx，完成下載!!!
---------------



## **👉3.以Python取得四大表、會計師查核報告、與財報附註**
 * 下方為公開資訊主要查詢網站
  * [公開資訊觀測站-案例文件整批下載:](https://mops.twse.com.tw/mops/web/t203sb02)
  去下載想要的年度

 * 主要參考影片如下:
  * [【Inline XBRL】以Python取得四大表、會計師查核報告、與財報附註](https://www.youtube.com/watch?v=oR5Phokt-aE)
  * [【Inline XBRL】以Python彙整所有公司資產負債表、損益表、與現金流量表](https://www.youtube.com/watch?v=Capp5fdGUQ0)
 * 甚麼是Inline XBRL?  

  * 即將傳統XBRL(xml檔)文件+用HTML編排=使其成為美觀易讀的財報Inline XBRL格式。  

  * 依證交所行政函釋，上市公司自2019年第1季起，財務報告除傳統pdf、word檔外，應在採用InlineXBR格式申報，並在財報公告截止日後15日內申報。


### **💡3-1.一次下載所有公司inline XBRL案件文件**

In [ ]:
#下載單一公司inline XBRL案件文件  #資料在4/16更新

#輸入想要的公司
company = int(input('輸入想要的公司代號'))

#確保目前路徑
import os
os.chdir('/content/drive/MyDrive/colab_cloud/finlab/inlineXBRL')

#建立新資料夾"公司基本資料查詢"，並將路徑移至資料夾
if not os.path.isdir(str(company)): # 確認是否有jobs_csv資料夾  沒有則返回Ture
  os.mkdir(str(company)) # 建立jobs_csv資料夾
  print(f'建立"{company}"夾完成')
else:
  print(f'已有{company}資料夾')
print('--------------------------')

file=f'./tifrs-fr1-m1-ci-cr-{company}-2021Q4.html'
with open(file,'r', encoding='utf-8') as f:
  iXBRL = f.read()
  df = pd.read_html(iXBRL)

os.chdir(f'./{company}')
df[0].to_excel(f'./資産負債表.xlsx')
df[1].to_excel(f'./綜合損益表.xlsx')
df[2].to_excel(f'./現金流量表.xlsx')
df[3].to_excel(f'./當期益動表.xlsx')
df[4].to_excel(f'./去年同期益動表.xlsx')
df[5].to_excel(f'./會計師查核報告.xlsx')
df[10].to_excel(f'./列入合併財務報表之子公司.xlsx')
df[14].to_excel(f'./應收款之帳齡分析.xlsx')
df[15].to_excel(f'./應收款之備抵損失變動資訊.xlsx')
df[17].to_excel(f'./關係人名稱及關係.xlsx')
df[29].to_excel(f'./主要管理階層薪酬.xlsx')
df[30].to_excel(f'./資金貸他人.xlsx')
df[31].to_excel(f'./為他人背書保證.xlsx')
df[32].to_excel(f'./母子公司間業務關係及重要交易往來情形.xlsx')
df[33].to_excel(f'./被投資公司名稱、所在地區…等相關資訊.xlsx')
df[34].to_excel(f'./大陸地區投資.xlsx')
print(f'完成{company}下載')

輸入想要的公司代號2330
已有2330資料夾
--------------------------
完成2330下載


In [ ]:
#一次下載所有公司inline XBRL案件文件

import shutil #可複製、移動、刪除檔案

os.chdir('/content/drive/MyDrive/colab_cloud/finlab/inlineXBRL')

source_folder = os.chdir('/content/drive/MyDrive/colab_cloud/finlab/inlineXBRL')
#建立新資料夾"公司基本資料查詢"，並將路徑移至資料夾
if not os.path.isdir('一般工商_合併報表'): # 確認是否有jobs_csv資料夾  沒有則返回Ture
  os.mkdir('一般工商_合併報表') # 建立jobs_csv資料夾
  print('建立"一般工商_合併報表"夾完成')
else:
  print('已有一般工商_合併報表資料夾')

target_folder = os.chdir('/content/drive/MyDrive/colab_cloud/finlab/inlineXBRL/一般工商_合併報表')

#快速移動資料夾

# 讀取資料夾中的檔案
for filename in os.listdir(source_folder):
    # 檔名的條件
    if 'fr1' in filename and 'ci' in filename and 'cr' in filename:
        #移動檔案到新資料夾
        shutil.move(os.path.join(source_folder, filename), target_folder)
        print('完成')

建立"一般工商_合併報表"夾完成


In [ ]:
os.chdir(folder_path)


In [ ]:
source_folder = 'tifrs'
target_folder = '一般工商_合併報表'

# 讀取資料夾中的檔案
for filename in os.listdir(source_folder):
    # 檔名的條件
    if 'fr1' in filename and 'ci' in filename and 'cr' in filename:
        #移動檔案到新資料夾
        shutil.move(os.path.join(source_folder, filename), target_folder)
        print(f'{filename} 正在移動中正在移動中')
print('移動完成')

移動完成


In [ ]:
# folder_path ='/content/drive/MyDrive/colab_cloud/finlab/inlineXBRL'
# for file in os.listdir(folder_path):
#     print(file)